# Load Required Libraries

In [3]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image
import random
import os


# Define the Dataset Path and Transformations

In [6]:
# Define the path to your dataset
dataset_path = '/Users/lennox/Documents/machineLearning/data/tenAnimalsImages'

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])


# Load the Dataset

In [9]:
# Load the dataset
dataset = ImageFolder(root=dataset_path, transform=transform)
class_names = dataset.classes


# Initialize DataLoader (Optional)

In [12]:
# Initialize DataLoader (optional)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


# Load the Model and Class Names

In [15]:
# Load the model and class names
model_load_path = '/Users/lennox/documents/machineLearning/research/builtModels/model_with_classes.pth'
model_data = torch.load(model_load_path, map_location=torch.device('cpu'))

# Retrieve the model state_dict and class names
model_state_dict = model_data['model_state_dict']
class_names = model_data['class_names']


/var/folders/n_/vx3kcgzj67q_z0t840cm_5ym0000gn/T/ipykernel_71645/3066097055.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_data = torch.load(model_load_path, map_

# Initialize the Model

In [18]:
# Initialize the model (ResNet18)
weights = models.ResNet18_Weights.DEFAULT
model = models.resnet18(weights=weights)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, len(class_names))  # Adjust the final layer to match the number of classes

# Load the saved state_dict
model.load_state_dict(model_state_dict)
model.eval()  # Set the model to evaluation mode


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# Define the Device

In [21]:
# Define the device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = model.to(device)


# Define the Prediction Function

In [24]:
# Function to predict the class of a given image
def predict_image(image_path, model, classes):
    model.eval()
    image = Image.open(image_path).convert('RGB')  # Ensure image is in RGB mode
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension and send to device
    output = model(image)
    _, predicted = torch.max(output.data, 1)
    return classes[predicted.item()]


# Select a Random Image from the Folder

In [27]:
# Select a random image from the folder
image_folder_path = '/Users/lennox/Documents/machineLearning/data/tenAnimalsImages'
random_image = random.choice(os.listdir(image_folder_path))
image_path = os.path.join(image_folder_path, random_image)


In [29]:
# Function to get a list of all image files in the dataset directory
def get_all_image_paths(dataset_path):
    image_paths = []
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths.append(os.path.join(root, file))
    return image_paths

# Get all image paths
all_image_paths = get_all_image_paths(dataset_path)

# Select a random image from the list
random_image_path = random.choice(all_image_paths)


# Predict the Class of the Random Image and Print Actual and Predicted Classes

In [32]:
# Predict the class of the random image
predicted_class = predict_image(random_image_path, model, class_names)

# Extract the actual class from the image path
actual_class = os.path.basename(os.path.dirname(random_image_path))

print(f'Image Path: {random_image_path}')
print(f'Actual Class: {actual_class}')
print(f'Predicted Class: {predicted_class}')

Image Path: /Users/lennox/Documents/machineLearning/data/tenAnimalsImages/chicken/503.jpeg
Actual Class: chicken
Predicted Class: horse


# Display the image

In [35]:
# Display the image
image = Image.open(random_image_path)
image.show()


In [39]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(224),
    transforms.ToTensor()
])


In [41]:
import torch
import torchvision.models as models

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 10)  # Adjust to your number of classes

# Freeze the layers except the final fully connected layer
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the final fully connected layer
for param in model.fc.parameters():
    param.requires_grad = True


/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [43]:
# Example: Adjust learning rate and number of epochs
optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)
num_epochs = 25


In [45]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 10)  # Adjust to your number of classes


/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [47]:
import torch.nn as nn

class CustomCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(64 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, num_classes)
    
    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv3(x))
        x = nn.functional.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CustomCNN(num_classes=10)


In [49]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [51]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(224),
    transforms.ToTensor()
])

# Load dataset
train_dataset = datasets.ImageFolder(root='/Users/lennox/Documents/machineLearning/data/tenAnimalsImages', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Load pre-trained model and modify
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 10)  # Adjust to your number of classes

# Freeze the layers except the final fully connected layer
for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True

# Move model to device
# Check if MPS is available
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f'Using device: {device}')
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Training loop
num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

print('Finished Training')


Using device: mps
Epoch 1/25, Loss: 0.5445, Accuracy: 0.8331
Epoch 2/25, Loss: 0.3816, Accuracy: 0.8770
Epoch 3/25, Loss: 0.3717, Accuracy: 0.8784
Epoch 4/25, Loss: 0.3646, Accuracy: 0.8790
Epoch 5/25, Loss: 0.3610, Accuracy: 0.8800
Epoch 6/25, Loss: 0.3657, Accuracy: 0.8799
Epoch 7/25, Loss: 0.3578, Accuracy: 0.8835
Epoch 8/25, Loss: 0.3441, Accuracy: 0.8866
Epoch 9/25, Loss: 0.3519, Accuracy: 0.8843
Epoch 10/25, Loss: 0.3474, Accuracy: 0.8856
Epoch 11/25, Loss: 0.3541, Accuracy: 0.8837
Epoch 12/25, Loss: 0.3526, Accuracy: 0.8845
Epoch 13/25, Loss: 0.3475, Accuracy: 0.8835
Epoch 14/25, Loss: 0.3466, Accuracy: 0.8860
Epoch 15/25, Loss: 0.3442, Accuracy: 0.8871
Epoch 16/25, Loss: 0.3432, Accuracy: 0.8865
Epoch 17/25, Loss: 0.3481, Accuracy: 0.8854
Epoch 18/25, Loss: 0.3428, Accuracy: 0.8852
Epoch 19/25, Loss: 0.3407, Accuracy: 0.8869
Epoch 20/25, Loss: 0.3452, Accuracy: 0.8869
Epoch 21/25, Loss: 0.3398, Accuracy: 0.8865
Epoch 22/25, Loss: 0.3452, Accuracy: 0.8840
Epoch 23/25, Loss: 0.33